<a href="https://colab.research.google.com/github/flohmannjr/PJI410/blob/main/SRAG_2021_a_2023S1_openDataSUS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SRAG 2021 a 2023 - Banco de Dados de Síndrome Respiratória Aguda Grave

Dados atualizados até 22 de Fevereiro de 2023

https://opendatasus.saude.gov.br/dataset/srag-2021-a-2023

# Importações

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import datetime

# Configurações

In [2]:
plt.rcParams['figure.figsize'] = [8, 5]
plt.rcParams['figure.dpi'] = 100

plt.style.use('seaborn-darkgrid')

SEED     = 2001088 + 2001247 + 2003061 + 2008193 + 2008620 + 2008880
COLORMAP = 'summer_r'

# Dados

In [3]:
!wget https://raw.githubusercontent.com/flohmannjr/PJI410/main/openDataSUS.7z.001
!wget https://raw.githubusercontent.com/flohmannjr/PJI410/main/openDataSUS.7z.002
!wget https://raw.githubusercontent.com/flohmannjr/PJI410/main/openDataSUS.7z.003
!wget https://raw.githubusercontent.com/flohmannjr/PJI410/main/openDataSUS.7z.004

!7z e /content/openDataSUS.7z.001

--2023-02-23 22:32:57--  https://raw.githubusercontent.com/flohmannjr/PJI410/main/openDataSUS.7z.001
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25165824 (24M) [application/octet-stream]
Saving to: ‘openDataSUS.7z.001’

openDataSUS.7z.001  100%[===================>]  24.00M   134MB/s    in 0.2s    

2023-02-23 22:32:57 (134 MB/s) - ‘openDataSUS.7z.001’ saved [25165824/25165824]

--2023-02-23 22:32:58--  https://raw.githubusercontent.com/flohmannjr/PJI410/main/openDataSUS.7z.002
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2

In [4]:
CAMPOS = {'DT_NOTIFIC': 'string', 'SEM_NOT': 'Int32', 'DT_SIN_PRI': 'string', 'SEM_PRI': 'Int32', 'SG_UF_NOT': 'string', 'ID_REGIONA': 'string', 'CO_REGIONA': 'Int32',
          'ID_MUNICIP': 'string', 'CO_MUN_NOT': 'Int32', 'ID_UNIDADE': 'string', 'CO_UNI_NOT': 'Int32', 'CS_SEXO': 'string', 'DT_NASC': 'string', 'NU_IDADE_N': 'Int32',
          'TP_IDADE': 'Int32', 'COD_IDADE': 'string', 'CS_GESTANT': 'Int32', 'CS_RACA': 'Int32', 'CS_ESCOL_N': 'Int32', 'ID_PAIS': 'string', 'CO_PAIS': 'Int32',
          'SG_UF': 'string', 'ID_RG_RESI': 'string', 'CO_RG_RESI': 'Int32', 'ID_MN_RESI': 'string', 'CO_MUN_RES': 'Int32', 'CS_ZONA': 'Int32', 'SURTO_SG': 'string',
          'NOSOCOMIAL': 'Int32', 'AVE_SUINO': 'Int32', 'FEBRE': 'Int32', 'TOSSE': 'Int32', 'GARGANTA': 'Int32', 'DISPNEIA': 'Int32', 'DESC_RESP': 'Int32', 'SATURACAO': 'Int32',
          'DIARREIA': 'Int32', 'VOMITO': 'Int32', 'OUTRO_SIN': 'Int32', 'OUTRO_DES': 'string', 'PUERPERA': 'Int32', 'FATOR_RISC': 'Int32', 'CARDIOPATI': 'Int32',
          'HEMATOLOGI': 'Int32', 'SIND_DOWN': 'Int32', 'HEPATICA': 'Int32', 'ASMA': 'Int32', 'DIABETES': 'Int32', 'NEUROLOGIC': 'Int32', 'PNEUMOPATI': 'Int32',
          'IMUNODEPRE': 'Int32', 'RENAL': 'Int32', 'OBESIDADE': 'Int32', 'OBES_IMC': 'string', 'OUT_MORBI': 'Int32', 'MORB_DESC': 'string', 'VACINA': 'Int32',
          'DT_UT_DOSE': 'string', 'MAE_VAC': 'Int32', 'DT_VAC_MAE': 'string', 'M_AMAMENTA': 'Int32', 'DT_DOSEUNI': 'string', 'DT_1_DOSE': 'string', 'DT_2_DOSE': 'string',
          'ANTIVIRAL': 'Int32', 'TP_ANTIVIR': 'Int32', 'OUT_ANTIV': 'string', 'DT_ANTIVIR': 'string', 'HOSPITAL': 'Int32', 'DT_INTERNA': 'string', 'SG_UF_INTE': 'string',
          'ID_RG_INTE': 'string', 'CO_RG_INTE': 'Int32', 'ID_MN_INTE': 'string', 'CO_MU_INTE': 'Int32', 'UTI': 'Int32', 'DT_ENTUTI': 'string', 'DT_SAIDUTI': 'string',
          'SUPORT_VEN': 'Int32', 'RAIOX_RES': 'Int32', 'RAIOX_OUT': 'string', 'DT_RAIOX': 'string', 'AMOSTRA': 'Int32', 'DT_COLETA': 'string', 'TP_AMOSTRA': 'Int32',
          'OUT_AMOST': 'string', 'PCR_RESUL': 'Int32', 'DT_PCR': 'string', 'POS_PCRFLU': 'Int32', 'TP_FLU_PCR': 'Int32', 'PCR_FLUASU': 'Int32', 'FLUASU_OUT': 'string',
          'PCR_FLUBLI': 'Int32', 'FLUBLI_OUT': 'string', 'POS_PCROUT': 'Int32', 'PCR_VSR': 'Int32', 'PCR_PARA1': 'Int32', 'PCR_PARA2': 'Int32', 'PCR_PARA3': 'Int32',
          'PCR_PARA4': 'Int32', 'PCR_ADENO': 'Int32', 'PCR_METAP': 'Int32', 'PCR_BOCA': 'Int32', 'PCR_RINO': 'Int32', 'PCR_OUTRO': 'Int32', 'DS_PCR_OUT': 'string',
          'CLASSI_FIN': 'Int32', 'CLASSI_OUT': 'string', 'CRITERIO': 'Int32', 'EVOLUCAO': 'Int32', 'DT_EVOLUCA': 'string', 'DT_ENCERRA': 'string', 'DT_DIGITA': 'string',
          'HISTO_VGM': 'string', 'PAIS_VGM': 'string', 'CO_PS_VGM': 'string', 'LO_PS_VGM': 'string', 'DT_VGM': 'string', 'DT_RT_VGM': 'string', 'PCR_SARS2': 'Int32',
          'PAC_COCBO': 'string', 'PAC_DSCBO': 'string', 'OUT_ANIM': 'string', 'DOR_ABD': 'Int32', 'FADIGA': 'Int32', 'PERD_OLFT': 'Int32', 'PERD_PALA': 'Int32',
          'TOMO_RES': 'Int32', 'TOMO_OUT': 'string', 'DT_TOMO': 'string', 'TP_TES_AN': 'Int32', 'DT_RES_AN': 'string', 'RES_AN': 'Int32', 'POS_AN_FLU': 'string',
          'TP_FLU_AN': 'string', 'POS_AN_OUT': 'Int32', 'AN_SARS2': 'Int32', 'AN_VSR': 'Int32', 'AN_PARA1': 'Int32', 'AN_PARA2': 'Int32', 'AN_PARA3': 'Int32',
          'AN_ADENO': 'Int32', 'AN_OUTRO': 'Int32', 'DS_AN_OUT': 'string', 'TP_AM_SOR': 'Int32', 'SOR_OUT': 'string', 'DT_CO_SOR': 'string', 'TP_SOR': 'Int32',
          'OUT_SOR': 'string', 'DT_RES': 'string', 'RES_IGG': 'Int32', 'RES_IGM': 'Int32', 'RES_IGA': 'Int32', 'ESTRANG': 'Int32', 'VACINA_COV': 'Int32',
          'DOSE_1_COV': 'string', 'DOSE_2_COV': 'string', 'DOSE_REF': 'string', 'FAB_COV_1': 'string', 'FAB_COV_2': 'string', 'FAB_COVREF': 'string', 'LAB_PR_COV': 'string',
          'LOTE_1_COV': 'string', 'LOTE_2_COV': 'string', 'LOTE_REF': 'string', 'FNT_IN_COV': 'Int32', 'DOSE_2REF': 'string', 'FAB_COVRF2': 'string', 'LOTE_REF2': 'string',
          'TRAT_COV': 'Int32', 'TIPO_TRAT': 'Int32', 'OUT_TRAT': 'string', 'DT_TRT_COV': 'string'}

# AUSENTES = df_23.columns[~df_23.columns.isin(df_21.columns)]
AUSENTES = ['DOSE_2REF', 'FAB_COVRF2', 'LOTE_REF2', 'TRAT_COV', 'TIPO_TRAT', 'OUT_TRAT', 'DT_TRT_COV']

DATAS = ['DT_NOTIFIC', 'DT_SIN_PRI', 'DT_NASC', 'DT_UT_DOSE', 'DT_VAC_MAE', 'DT_DOSEUNI', 'DT_1_DOSE', 'DT_2_DOSE', 'DT_ANTIVIR', 'DT_INTERNA', 'DT_ENTUTI', 'DT_SAIDUTI',
         'DT_RAIOX', 'DT_COLETA', 'DT_PCR', 'DT_EVOLUCA', 'DT_ENCERRA', 'DT_DIGITA', 'DT_VGM', 'DT_RT_VGM', 'DT_TOMO', 'DT_RES_AN', 'DT_CO_SOR', 'DT_RES', 'DT_TRT_COV']

In [5]:
df_srag = pd.read_csv('INFLUD21-20-02-2023.csv', sep=';', dtype=dict(list(CAMPOS.items())[:-7]))

In [6]:
df_srag[AUSENTES] = [None] * len(AUSENTES)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


In [7]:
df_srag = pd.concat([df_srag, pd.read_csv('INFLUD22-13-02-2023.csv', sep=';', dtype=CAMPOS)])
df_srag = pd.concat([df_srag, pd.read_csv('INFLUD23-13-02-2023.csv', sep=';', dtype=CAMPOS)])

In [8]:
for campo in DATAS:
    df_srag[campo] = pd.to_datetime(df_srag[campo], format='%d/%m/%Y', errors='coerce')

In [9]:
df_srag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2310360 entries, 0 to 22642
Columns: 173 entries, DT_NOTIFIC to DT_TRT_COV
dtypes: Int32(97), datetime64[ns](25), string(51)
memory usage: 2.4 GB


In [10]:
df_srag.info(max_cols=180, null_counts=True)

<ipython-input-10-0c5653319908>:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df_srag.info(max_cols=180, null_counts=True)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2310360 entries, 0 to 22642
Data columns (total 173 columns):
 #    Column      Non-Null Count    Dtype         
---   ------      --------------    -----         
 0    DT_NOTIFIC  2310360 non-null  datetime64[ns]
 1    SEM_NOT     2310360 non-null  Int32         
 2    DT_SIN_PRI  2310360 non-null  datetime64[ns]
 3    SEM_PRI     2310360 non-null  Int32         
 4    SG_UF_NOT   2310360 non-null  string        
 5    ID_REGIONA  2050155 non-null  string        
 6    CO_REGIONA  2050155 non-null  Int32         
 7    ID_MUNICIP  2310360 non-null  string        
 8    CO_MUN_NOT  2310360 non-null  Int32         
 9    ID_UNIDADE  2310360 non-null  string        
 10   CO_UNI_NOT  2310360 non-null  Int32         
 11   CS_SEXO     2310360 non-null  string        
 12   DT_NASC     2308970 non-null  datetime64[ns]
 13   NU_IDADE_N  2310360 non-null  Int32         
 14   TP_IDADE    2310360 non-null  Int32         
 15   COD_IDADE   231